In [ ]:
%%writefile cudaprimes.cu
#include <stdio.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define NUM_BLOCKS    10
#define NUM_THREADS   100
#define TOTAL_THREADS (NUM_BLOCKS * NUM_THREADS)
#define TOTAL_PRIMES  1000000

#define PRIMES_PER_THREAD (TOTAL_PRIMES / TOTAL_THREADS)

//helper function to check for prime
__device__ bool isPrime(int n)
{
  if(n < 2) return false; // for 0 and 1
  if  (n == 2 || n == 3) return true; //for 2 and 3
  if (n % 2 == 0) return false; //for even numbers

  int limit = sqrtf(n) + 1;
  for(int i = 3; i < limit; i += 2)
  {
    if(n % i == 0) return false;
  }
  return true;
}

__global__ void findPrimesKernel(int *d_primes){
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int baseIndex = tid * PRIMES_PER_THREAD;
  int count = 0;
  int value = 2;

  while (count < PRIMES_PER_THREAD)
  {
    if(isPrime(value))
    {
      d_primes[baseIndex + count] = value;
      count++;
    }
    value++;
  }
}

int main()
{
  int *h_primes;
  int *d_primes;

  const int size = TOTAL_PRIMES * sizeof(int);

  //memory for host
  h_primes = (int *)malloc(size);

  //memory for device
  cudaMalloc((void **)&d_primes, size);

  //launch kernel
  findPrimesKernel<<<10, 100>>>(d_primes);

  //wait for GPU to finish
  cudaDeviceSynchronize();

  //copy back to host
  cudaMemcpy(h_primes, d_primes, size, cudaMemcpyDeviceToHost);

  //print result
  for(int i = 0; i < 1000; i++){
    printf("prime[%d] = %d\n" , i+1, h_primes[i]);
  }

  cudaFree(d_primes);
  free(h_primes);

  return 0;

}

Overwriting cudaprimes.cu


In [ ]:
!nvcc cudaprimes.cu -o cudaprimes -arch=sm_70

In [ ]:
!./cudaprimes

prime[1] = 2
prime[2] = 3
prime[3] = 5
prime[4] = 7
prime[5] = 11
prime[6] = 13
prime[7] = 17
prime[8] = 19
prime[9] = 23
prime[10] = 29
prime[11] = 31
prime[12] = 37
prime[13] = 41
prime[14] = 43
prime[15] = 47
prime[16] = 53
prime[17] = 59
prime[18] = 61
prime[19] = 67
prime[20] = 71
prime[21] = 73
prime[22] = 79
prime[23] = 83
prime[24] = 89
prime[25] = 97
prime[26] = 101
prime[27] = 103
prime[28] = 107
prime[29] = 109
prime[30] = 113
prime[31] = 127
prime[32] = 131
prime[33] = 137
prime[34] = 139
prime[35] = 149
prime[36] = 151
prime[37] = 157
prime[38] = 163
prime[39] = 167
prime[40] = 173
prime[41] = 179
prime[42] = 181
prime[43] = 191
prime[44] = 193
prime[45] = 197
prime[46] = 199
prime[47] = 211
prime[48] = 223
prime[49] = 227
prime[50] = 229
prime[51] = 233
prime[52] = 239
prime[53] = 241
prime[54] = 251
prime[55] = 257
prime[56] = 263
prime[57] = 269
prime[58] = 271
prime[59] = 277
prime[60] = 281
prime[61] = 283
prime[62] = 293
prime[63] = 307
prime[64] = 311
prime[65] = 31